<div style="background-color: #FFDDDD; border-left: 5px solid red; padding: 10px; color: black;">
    <strong>Kernel:</strong> Python 3 (ipykernel)
</div>

# 🚀 Deploy `deepseek-ai/DeepSeek-R1-Distill-Llama-8B` on Amazon SageMaker

To start off, let's install some packages to help us through the notebooks. **Restart the kernel after packages have been installed.**

In [7]:
%pip install -r ./scripts/requirements.txt --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 241.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 97.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 75.0 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.45.1
    Uninstalling bitsandbytes-0.45.1:
      Successfully uninstalled bitsandbytes-0.45.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.3.0
    Uninstalling accelerate-1.3.0:
      Successfully uninstalled accelerate-1.3.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.2
    Uninstalling transformers-4.48.2:
      Successfully uninstalled transformers-4.48.2
  Attempting uninstall: datasets
    Found existing installation: datasets 3.2.0
    Uninstalling datasets-3.2.0:
      Successfully uninstalled datasets-3.2.0
  Attempting uninstall: trl
    Found existing installation: trl 0.9

In [1]:
import os
import sagemaker
from sagemaker.djl_inference import DJLModel
from ipywidgets import Dropdown

import sys
sys.path.append(os.path.dirname(os.getcwd()))

from utilities.helpers import (
    pretty_print_html, 
    set_meta_llama_params,
    print_dialog,
    format_messages,
    write_eula
)

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
session = sagemaker.Session()
role = sagemaker.get_execution_role()
default_bucket = session.default_bucket()

print(f"Execution Role: {role}")
print(f"Default S3 Bucket: {default_bucket}")

Execution Role: arn:aws:iam::058264176820:role/amazon-sagemaker-base-executionrole
Default S3 Bucket: sagemaker-us-east-1-058264176820


## Deploy Model to SageMaker Hosting

### Step 1: Get SageMaker LMI Container to host DeepSeek

In [5]:
inference_image_uri = sagemaker.image_uris.retrieve(
    framework="djl-lmi", 
    region=session.boto_session.region_name, 
    version="latest"
)
pretty_print_html(f"using image to host: {inference_image_uri}")

[04/06/25 21:07:46] INFO     Ignoring unnecessary instance type: None.                            ]8;id=562372;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=447721;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#530\530]8;;\

### Step 2: Deploy model using `DJLModel`

In [11]:
inference_llm_config = {
    "HF_MODEL_ID": "deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    "HF_API_TOKEN":"",
    "OPTION_MAX_MODEL_LEN": "4096",
    "OPTION_GPU_MEMORY_UTILIZATION": "0.8",
    "OPTION_ENABLE_STREAMING": "false",
    "OPTION_ROLLING_BATCH": "auto",
    "OPTION_MODEL_LOADING_TIMEOUT": "3600",
    # "OPTION_OUTPUT_FORMATTER": "jsonlines",
    "OPTION_PAGED_ATTENTION": "false",
    "OPTION_DTYPE": "fp16",
}

In [12]:
model_name = "DeepSeek-R1-Distill-Llama-8B"

lmi_model = sagemaker.Model(
    image_uri=inference_image_uri,
    env=inference_llm_config,
    role=role,
    name=model_name
)

In [ ]:
base_endpoint_name = f"{model_name}-endpoint"

predictor = lmi_model.deploy(
    initial_instance_count=1, 
    instance_type="ml.g5.2xlarge",
    endpoint_name=base_endpoint_name
)

[04/06/25 21:49:55] INFO     Creating model with name: DeepSeek-R1-Distill-Llama-8B                 ]8;id=620378;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=787772;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4094\4094]8;;\

[04/06/25 21:49:56] WARNING  Using already existing model: DeepSeek-R1-Distill-Llama-8B             ]8;id=33337;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=948292;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4105\4105]8;;\

                    INFO     Creating endpoint-config with name                                     ]8;id=288588;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=74033;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#6019\6019]8;;\
                             DeepSeek-R1-Distill-Llama-8B-endpoint                                                 

                    INFO     Creating endpoint with name DeepSeek-R1-Distill-Llama-8B-endpoint      ]8;id=833370;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=544766;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4841\4841]8;;\

------------------------------------------

## Run Inference

In [ ]:
import sagemaker
from datasets import load_dataset

In [ ]:
sagemaker_session = sagemaker.Session()

In [ ]:
model_id = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

endpoint_name = base_endpoint_name

In [ ]:
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)

In [ ]:
def create_summarization_prompts(data_point):
    full_prompt =f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
                    You are an AI assistant trained to summarize conversations. Provide a concise summary of the dialogue, capturing the key points and overall context.
                    <|eot_id|><|start_header_id|>user<|end_header_id|>
                    Summarize the following conversation:

                    {data_point["dialogue"]}
                    <|eot_id|><|start_header_id|>assistant<|end_header_id|>
                    Here's a concise summary of the conversation in a single sentence:

                    <|eot_id|>"""
    return {"prompt": full_prompt}

In [ ]:
from pprint import pprint
# HF dataset that we will be working with 
dataset_name="Samsung/samsum"
    
# Load dataset from the hub
dataset = load_dataset(dataset_name, split="test")

random_row = dataset.shuffle().select(range(1))[0]

random_prompt=create_summarization_prompts(random_row)
pprint(random_prompt)

In [ ]:
response = predictor.predict(
    {
        "inputs": random_prompt['prompt'],
        "parameters": {
            "do_sample":True,
            "max_new_tokens":200,
            "top_p":0.95,
            "top_k":50,
            "temperature":0.7,
            "stop": ['<|eot_id|>', '<|end_of_text|>']
        },
    }
)

response['generated_text']